# 5. Recurrent Neural Network
## 1. About Recurrent Neural Network
Feedforward Neural Networks Transition to Recurrent Neural Networks
- RNN are essentially FFNN where the non-linear output of a hidden layer is used as input to the same hidden layer through an extra linear function and it will be summed to the input at that time point.
- The resulting effect is that every input **reverberates** in the network for more time (like a short term memory).
- This makes the network sensitive to the temporal structure of the input (order and interval)

## Model A
One hidden layer with ReLU
- unroll 28 time steps
    - each step input size: 28 x 1
    - total per unroll: 28 x 28
        - Feedforward Neural Network input size: 28 x 28
- 1 Hidden layer
- ReLU Activation function

In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.rnn = nn.RNN(input_dim, 
                          hidden_dim, 
                          layer_dim, 
                          batch_first=True, 
                          nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        
        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28
hidden_dim = 100
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 10

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    model.cuda()
    
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

'''
STEP 7: TRAIN THE MODEL
'''

# Number of steps to unroll
seq_dim = 28  

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)
            
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1, seq_dim, input_dim))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 1.2869062423706055. Accuracy: 53
Iteration: 1000. Loss: 0.6870487928390503. Accuracy: 72
Iteration: 1500. Loss: 0.6972125768661499. Accuracy: 79
Iteration: 2000. Loss: 0.7116541266441345. Accuracy: 81
Iteration: 2500. Loss: 0.4290916323661804. Accuracy: 90
Iteration: 3000. Loss: 0.25698167085647583. Accuracy: 91
Iteration: 3500. Loss: 0.18241305649280548. Accuracy: 91
Iteration: 4000. Loss: 0.19922585785388947. Accuracy: 91
Iteration: 4500. Loss: 0.24579021334648132. Accuracy: 93
Iteration: 5000. Loss: 0.13444818556308746. Accuracy: 92
Iteration: 5500. Loss: 0.2651044726371765. Accuracy: 92
Iteration: 6000. Loss: 0.18801598250865936. Accuracy: 94
Iteration: 6500. Loss: 0.17202286422252655. Accuracy: 92
Iteration: 7000. Loss: 0.12266672402620316. Accuracy: 89
Iteration: 7500. Loss: 0.29131200909614563. Accuracy: 89
Iteration: 8000. Loss: 0.23031577467918396. Accuracy: 94
Iteration: 8500. Loss: 0.08916956931352615. Accuracy: 93
Iteration: 9000. Loss: 0.1504618376493

In [9]:
images.size()


torch.Size([100, 28, 28])

In [10]:
nn.RNN(input_dim, 
                          hidden_dim, 
                          layer_dim, 
                          batch_first=True, 
                          nonlinearity='relu')

RNN(28, 100, batch_first=True)

In [11]:
a = nn.RNN(input_dim, 
                          hidden_dim, 
                          layer_dim, 
                          batch_first=True, 
                          nonlinearity='relu')


In [12]:
a


RNN(28, 100, batch_first=True)